In [1]:
#Importing packages

import numpy as np
import pandas as pd
import requests
import glob
import openpyxl
import os

Creating relative paths

In [125]:
#creating relative paths
votes = os.path.join("VoteWatch-EP-voting-data_2004-2022", "EP6_RCVs_2022_06_13.xlsx")
votations = os.path.join("VoteWatch-EP-voting-data_2004-2022","EP6_Voted docs.xlsx")


Importing data from EU API (for correct names)

In [126]:

url = 'https://data.europarl.europa.eu/api/v2/meps?parliamentary-term=6&format=application%2Fld%2Bjson&offset=0'
response = requests.get(url)
meps_data = response.json()

Creating DataFrames (runtime is quite long, .xlsx files)

In [146]:

meps_df_6 = pd.json_normalize(meps_data['data'])
EP_6_votes = pd.read_excel(votes)
EP_6_votations = pd.read_excel(votations)

KeyboardInterrupt: 

Inspecting the data

In [128]:
meps_df_6.head()

,id,type,identifier,label,familyName,givenName,sortLabel,officialFamilyName,officialGivenName
0,person/2278,Person,2278,Paul RÜBIG,Rübig,Paul,RUBIG,NaN,NaN
1,person/28196,Person,28196,Irena BELOHORSKÁ,Belohorská,Irena,BELOHORSKA,NaN,NaN
2,person/4436,Person,4436,Gianni PITTELLA,Pittella,Gianni,PITTELLA,NaN,NaN
3,person/39713,Person,39713,Renate WEBER,Weber,Renate,WEBERRENATE,NaN,NaN
4,person/34482,Person,34482,Alexandru-Ioan MORȚUN,Morțun,Alexandru-Ioan,MORTUN,NaN,NaN


Dropping irrelevant columns (to discuss)

In [129]:
meps_df_6.drop(columns=['id','type','sortLabel','officialFamilyName','officialGivenName'],inplace=True)

Renaming variables for pd.merge and correcting datatype

In [130]:
meps_df_6.rename(columns={'identifier': 'MepId'},inplace=True)
EP_6_votes.rename(columns={'WebisteEpID': 'MepId'},inplace=True)
meps_df_6.MepId = meps_df_6.MepId.astype(int)

Merging 

In [131]:
EP_6_votes_fixed_names = pd.merge(EP_6_votes,meps_df_6 , on='MepId', how='left')

Updating names of MPs

In [132]:
EP_6_votes_fixed_names[['Fname','Lname','FullName']] = EP_6_votes_fixed_names[['givenName','familyName','label']]
EP_6_votes_fixed_names.drop(columns=['label','familyName','givenName'],inplace=True)

Encoding missing party names

In [133]:
EP_6_votes_fixed_names.Party = EP_6_votes_fixed_names.Party.replace({".":np.NAN,"--":np.NAN})

Votations dataset inspection

In [134]:
EP_6_votations.head()

,euro_act_id,date,title,procedure,reading,type_of_vote,type_of_vote_en,rule,raporteur,url,...,final_vote,author_name,am_no,result_code,yes,no,abstain,did_not_vote,absent,absent_motivated
0,1,2004-09-15 00:00:00,EU-Mauritius fishing agreement,CNS *,Leg,Amended proposal for a Council regulation,Amended proposal for a Council regulation,s,Morillon Philippe,http://www.europarl.europa.eu/sides/getDoc.do?...,...,0,0,NaN,+,405,63.0,73.0,0,0,0
1,2,2004-09-15 00:00:00,EU-Mauritius fishing agreement,CNS *,Leg,Legislative resolution for a Council regulation,Legislative resolution for a Council regulation,s,Morillon Philippe,http://www.europarl.europa.eu/sides/getDoc.do?...,...,1,0,NaN,+,415,56.0,67.0,0,0,0
2,3,2004-09-15 00:00:00,EU-Madagascar Fishing agreement,CNS *,Leg,Amended proposal for a Council regulation,Amended proposal for a Council regulation,s,Morillon Philippe,http://www.europarl.europa.eu/sides/getDoc.do?...,...,0,0,NaN,+,504,79.0,84.0,0,0,0
3,4,2004-09-15 00:00:00,EU-Madagascar Fishing agreement,CNS *,Leg,Legislative resolution for a Council regulation,Legislative resolution for a Council regulation,s,Morillon Philippe,http://www.europarl.europa.eu/sides/getDoc.do?...,...,0,0,NaN,+,507,68.0,86.0,0,0,0
4,5,2004-09-15 00:00:00,EU-Cape Verde Fishing agreement,CNS *,Leg,Amended proposal for a Council regulation,Amended proposal for a Council regulation,s,Morillon Philippe,http://www.europarl.europa.eu/sides/getDoc.do?...,...,0,0,NaN,+,495,74.0,104.0,0,0,0


Dropping irrelevant columns (also to be discussed)

In [135]:
EP_6_votations_columnsdropped = EP_6_votations.drop(columns=['type_of_vote_en', 'did_not_vote','absent','absent_motivated'])

Manually fixing some dates, in order for pd.to_datetime to work

In [136]:
replacements = {
    " 00:00:00": "",
    " ian ":"/01/",
    "03/11/2008":"11/03/2008",
    "03/12/2008":"12/03/2008",
    "13/12/2009":"13/12/2007",
    "13/12/2008":"13/12/2007",
    "3/13/2008":"13/3/2008",
    '2009-12-13':'2007-12-13',
}
EP_6_votations_copy = EP_6_votations_columnsdropped.copy()

EP_6_votations_copy['date'] = EP_6_votations_copy['date'].astype(str).replace(replacements, regex=True)

In [137]:
EP_6_votations_copy.date

0       2004-09-15
1       2004-09-15
2       2004-09-15
3       2004-09-15
4       2004-09-15
           ...    
6194     7.05.2009
6195     7.05.2009
6196     7.05.2009
6197     7.05.2009
6198     7.05.2009
Name: date, Length: 6199, dtype: object

In [138]:
EP_6_votations_date_fixed = EP_6_votations_copy.copy()

In [139]:
EP_6_votations_date_fixed['date'] = pd.to_datetime(EP_6_votations_copy['date'],format="mixed",dayfirst=True,yearfirst=True) 

In [140]:
EP_6_votations_date_fixed.date.max()

Timestamp('2009-05-07 00:00:00')

Recoding columns in line with convention for the app


In [141]:
original_columns = EP_6_votations_date_fixed.columns.tolist()

new_columns = [
    "VoteId",
    "Date",
    "Title",
    "Procedure",
    "Leg/Non-Leg/Bud",
    "TypeOfVote",
    "VotingRule",
    "Rapporteur",
    "Link",
    "CommitteeResponsabile",
    "PolicyArea",
    "Subject",
    "FinalVote",
    "Author",
    "AmNo",
    "Vote",
    "Yes",
    "No",
    "Abs"
]

# Create a dictionary mapping original column names to new names
rename_dict = dict(zip(original_columns, new_columns))

# Assuming 'df' is your DataFrame
EP_6_votations_column_names_good = EP_6_votations_date_fixed.rename(columns=rename_dict)

Recoding missing data

In [142]:
def recode_votations(df):
    df['TypeOfVote'] = df['TypeOfVote'].replace(0,np.NAN)
    df['VotingRule'] = df['VotingRule'].replace(0,np.NAN)
    df['Rapporteur'] = df['Rapporteur'].replace(0,np.NAN)
    df['CommitteeResponsabile'] = df['CommitteeResponsabile'].replace(0,np.NAN)
    df['Author'] = df['Author'].replace(0,np.NAN)
    df['Subject'] = df['Subject'].replace(0,np.NAN)
    df['AmNo'] = df['AmNo'].replace(0,np.NAN)
    df['Vote'] = df['Vote'].replace({'+': 1, '-': 0})
    df['Vote'] = df['Vote'].replace({"-*":0," +":1, "?  replaced by 1043":0})



In [143]:
recode_votations(EP_6_votations_column_names_good)

C:\Users\zwiad\AppData\Local\Temp\ipykernel_7392\3001296997.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Vote'] = df['Vote'].replace({"-*":0," +":1, "?  replaced by 1043":0})


Checking and fixing the datatypes

In [152]:
EP_6_votations_column_names_good.dtypes

VoteId                            int64
Date                     datetime64[ns]
Title                            object
Procedure                        object
Leg/Non-Leg/Bud                  object
TypeOfVote                       object
VotingRule                       object
Rapporteur                       object
Link                             object
CommitteeResponsabile            object
PolicyArea                       object
Subject                          object
FinalVote                         int64
Author                           object
AmNo                             object
Vote                              Int64
Yes                               Int64
No                                Int64
Abs                               Int64
dtype: object

In [149]:
EP_6_votations_column_names_good.No = EP_6_votations_column_names_good['No'].astype('Int64')
EP_6_votations_column_names_good.Abs = EP_6_votations_column_names_good['Abs'].astype('Int64')
EP_6_votations_column_names_good.Vote = EP_6_votations_column_names_good['Vote'].astype('Int64')
EP_6_votations_column_names_good.Yes = EP_6_votations_column_names_good['Yes'].astype('Int64')

Renaming EPG groups to fit the API convention

In [154]:

EPG_rename_dict = {
    "Group of the European People's Party (Christian Democrats) and European Democrats":"EPP-ED",
    "Socialist Group in the European Parliament":"PES",
    "Group of the Alliance of Liberals and Democrats for Europe":"ALDE",
    "Union for Europe of the Nations Group":"UEN",
    "Confederal Group of the European United Left - Nordic Green Left":"GUE–NGL",
    "Group of the Greens/European Free Alliance":"Greens–EFA",
    "Non-attached Members":"NI",  
    "Independence/Democracy Group":"IND/DEM"
}

In [155]:
EP_6_votes_fixed_names.EPG = EP_6_votes_fixed_names.EPG.replace(EPG_rename_dict, regex=False)


In [156]:
EP_6_votes_fixed_names.EPG.value_counts()


EPG
EPP-ED        341
PES           263
ALDE          126
UEN            51
GUE–NGL        48
Greens–EFA     44
NI             40
IND/DEM        27
Name: count, dtype: int64

Creating a list to filter the non,observation columns from Votes


In [159]:
filtered_list = ['MepId',
 'Fname',
 'Lname',
 'FullName',
 'Activ',
 'Country',
 'Party',
 'EPG',
 'Start',
 'End']

Melting votes into long format

In [160]:

mep_info = EP_6_votes_fixed_names[filtered_list]
vote_columns = [col for col in EP_6_votes_fixed_names.columns if col not in filtered_list]
melted_votes = pd.melt(EP_6_votes_fixed_names, id_vars='MepId', value_vars=vote_columns, var_name='VoteId', value_name='Vote')


In [192]:
melted_votes.VoteId = melted_votes.VoteId.astype('Int64')

Adding EP_ID

In [193]:
melted_votes['EP_ID'] = 6
EP_6_votations_column_names_good['EP_ID'] = 6
mep_info['EP_ID'] = 6

C:\Users\zwiad\AppData\Local\Temp\ipykernel_7392\2793180917.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mep_info['EP_ID'] = 6


Saving files to .csv

In [194]:
base_directory = os.path.join("Cleaned_data","EP6_clean_data")
os.makedirs(base_directory, exist_ok=True)


In [195]:
mep_info.to_csv(os.path.join(base_directory,"mep_info_EP_6.csv"),index=False)
melted_votes.to_csv(os.path.join(base_directory,"votes_EP_6.csv"),index=False)
EP_6_votations_column_names_good.to_csv(os.path.join(base_directory,"votations_EP_6.csv"),index=False)
